## Notebook: Contextual Anomaly Detection (CAD) Device Creation

This is the second of a series of three notebooks that show how the Contextual Anomaly Detection (CAD) Accelerator can be used to train and deploy a prediction interval model into Monitor using the Model Factory service endpoints.
1.   Training: cookbooks/contextual_anomaly_train.ipynb
2.  Monitor Device Creation: cookbooks/contextual_anomaly_create_device.ipynb
3.   Model Deployment: cookbooks/contextual_anomaly_deploy_model.ipynb


### CAD Description

The CAD training job produces three prediction interval models that capture the normal operation (non-anomalous) behaviour of a given target variable based on a set of input features. 

These are point estimate multivariate regression machine learning models (denoted as base regressor) enhanced with conformal prediction statistical wrappers to produce a lower and upper bound that contain the target variable with probability 95% under normal operation (non-anomalous) conditions. Therefore, the probability of observing a target variable outside of the provided interval is 5% under normal operation conditions.

### Wind Turbine Dataset Description: 

In this notebook we use the CAD Accelerator to learn a prediction interval model that covers the normal operation behaviour of the Average Reactive Power of a Wind turbine asset with probability 95%. We consider 4 input features to predict the target variable. These are Average Active Power, Average Generator Bearing 1 Temperature, Average Generator Bearing 2 Temperature, Average Wind Speed.

### Device Creation Notebook Description

This notebook shows an example of how the Model Factory Service can be used to create a monitor device needed before deploying the trained model. The device created in Monitor will contain as metrics the different variables of the dataset (features,target and time stamp).


This is the SECOND notebook of our tutorial and should be run after 'cookbooks/contextual_anomaly_train.ipynb' since it requires the model specification file generated at the end of the CAD training notebook.





<a id='notebook_workflow'></a>
### Notebook Workflow
- [Imports](#imports)
- [Load KPI specification file (yaml)](#load_kpiyaml)
- [Load Model Factory config file (yaml)](#load_mfyaml)
- [Load Model specification file (yaml)](#load_modelspecyaml)
- [Prepare Payload to Create Monitor Device](#prepare_payload)
- [Post Create Device Job ](#create_device_post)
- [Request job log and summary](#job_log)

<a id='imports'></a>
### Imports

In [ ]:
import requests
import yaml
import pandas as pd

<a id='load_kpiyaml'></a>
### Load KPI specification file (yaml)


This file is a dictionary containing:

    - asset_id_column: column corresponding to asset id
    - data_name: train dataset file name
    - device_description: device description (optional)
    - mas_device_name: monitor device name (needed for device creation and/or deployment)
    - feature_columns: feature columns name as in first row of dataset csv file separated by ',' (e.g., P_avg,Rs_avg,Gb1t_avg,Ws_avg)
    - feature_names: feature columns interpretable names separated by ',' (same order as in feature_columns)
    - target_columns: target column name as in first row of dataset
    - target_names: target column interpretable name
    - timestamp_column: time stamp column name as in first row of dataset
    - timestamp_format: '%m/%d/%Y %H:%M'
    - inference_data_name: test dataset file name (optional for inference)
    - feature_map: dictionary mapping feature columns to descriptions (optional)

In [ ]:
input_file_name = "../config/Pavg_kpi.yml"

with open(input_file_name, 'r') as file:
    input_data = yaml.safe_load(file)

print('KPI specification file: ')
print(input_data)

<a id='load_mfyaml'></a>
### Load Model Factory config file (yaml)


This file is a dictionary containing:

    - endpoint_url: <ACTION: Replace with Model Factory endpoint URL>
    - train_recipe_endpoint: recipe/supervised-anomaly (DON'T CHANGE)
    - deploy_recipe_endpoint: deployment/monitor/model/create (DON'T CHANGE)
    - create_device_recipe_endpoint: deployment/monitor/device/create (DON'T CHANGE)

In [ ]:
model_factory_config_file_name = "../config/model_factory_config.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

<a id='load_modelspecyaml'></a>
### Load Model specification file (yaml)

This is the .yaml file saved in the last step of the contextual_anomaly_train.ipynb notebook. It is a dictionary containing the following information:

    - onnx_model_uri : Model mlflow uri provided in the CAD training summary
    - train_job_id : Model Factory CAD training job id
    - mas_device_name : Name of the Monitor Device

In [ ]:
input_file_name = "../config/model_info.yml"

with open(input_file_name, 'r') as file:
    model_info_data = yaml.safe_load(file)

print(model_info_data)

<a id='prepare_payload'></a>
### Prepare Payload to Create Monitor Device

The create device payload requires the following files:

- iot_credentials: This file is located in config/iot_credentials.yaml and is a dictionary containing the following information
    - IOT_URL: <INSERT>
    - IOT_API_KEY: <INSERT>
    - IOT_API_TOKEN: <INSERT>
    - model_api: /api/v0002/pipeline/models
    - MAS_URL: <INSERT>
    - X-api-key: <INSERT>
    - X-api-token: <INSERT>
    - mam_user_email: <INSERT>
    - tenantId: <INSERT>


- data_file: This file is a .csv file containing a single row of data (i.e., first two rows of ../data/Wind_Turbine_train.csv). In this example we use the file located in '../data/Wind_Turbine_Asset_Data.csv'

- assetmodel_file: This file is a .csv file containing 4 columns corresponding to dimension (name of variable/feature), type, label and columntype (following Monitor convention). In this example we use the file located in '../data/Wind_Turbine_Asset_Model.csv'

- device_type_name: Name of mas device (this will be obtained from the Model Specification file, [see above](#load_modelspecyaml) )

- device_type_description: Text description of the MAS device

- metric_columns: The metric associated with the device and available as columns in the data_file.

- timestamp_column: Time stamp column name (this will be obtained from the KPI specification file, [see above](#load-kpi-specification-file-yaml)).

- assetid_column: Asset id column name (this will be obtained from the KPI specification file, [see above](#load-kpi-specification-file-yaml)).

In [ ]:
'''
Credential File Path and Name
'''

credentials_file_path = "../config/iot_credentials_beta.yaml"
credentials_file_name = "iot_credentials_beta.yaml"

'''
Data File Path and Name
'''

data_file_create_device_path = "../data/Wind_Turbine_Asset_Data.csv"
data_file_create_device_name = "Wind_Turbine_Asset_Data.csv"

'''
assetmodel File Path and Name
'''

assetmodel_file_create_device_path = "../data/Wind_Turbine_Asset_Model.csv"
assetmodel_file_create_device_name = "Wind_Turbine_Asset_Model.csv"


'''
Device Type Name and Description
'''

device_type_name = model_info_data['mas_device_name']
device_type_description= 'Wind Turbine Device'

'''
Columns
'''

metric_columns = pd.read_csv(assetmodel_file_create_device_path)['dimension'].unique()
metric_columns = ",".join(metric_columns)

timestamp_column = input_data['timestamp_column']
assetid_column = input_data['asset_id_column']

In [ ]:
payload = {}
payload["device_type_name"] = device_type_name
payload["device_type_description"] = device_type_description
payload["metric_columns"] = metric_columns
payload["timestamp_column"] = timestamp_column
payload["assetid_column"] = assetid_column


files = [
    ("iot_credentials", (credentials_file_name, open(credentials_file_path), "text/csv")),
    ("data_file", (data_file_create_device_name, open(data_file_create_device_path), "text/csv")),
    ("assetmodel_file", (assetmodel_file_create_device_name, open(assetmodel_file_create_device_path), "text/csv")),

]

In [ ]:
print(payload)

<a id='create_device_post'></a> 
### Post Create Device Job 

In [ ]:
endpoint_url = model_factory_config["endpoint_url"]
create_device_url = endpoint_url + model_factory_config["create_device_recipe_endpoint"]
print(create_device_url)

In [ ]:
headers = {
  'accept': 'application/json'
}
response = requests.post(create_device_url, data=payload, files=files, headers=headers)

post_r_json = response.json()
print(post_r_json)
job_id = post_r_json['job_id']
print('job id:',job_id )

<a id='job_log'></a> 
### Request job log and summary

In [ ]:
logs = requests.get(endpoint_url + "log/" + job_id, headers=headers)
print('LOGS :: ')
if "logs" in logs.json():
    print(logs.json()["logs"])
else:
    print(logs.json())

summary = requests.get(endpoint_url + "summary/" + job_id, headers=headers)
print('SUMMARY :: ')
print(summary.json())